<a href="https://colab.research.google.com/github/JuanCervigon/Globalizacion/blob/main/Agrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

A partir de la tabla de Figaro en formato largo csv, se agregan ramas de actividad y se guarda el resultado en un excel en formato plano y otro en formato matricial

In [ ]:
# Importar librerías necesarias

import pandas as pd

# Importar datos

Importar los datos (drag and drop)

* Mapeo de los agregados originales a los finales

* Tabla flat Figaro

# Mapeo de los agregados

Crear un diccionario con mapeo de los agregados originales a los agregados definidos

* Leer un excel con el mapeo de los valores originales a los valores agregados y guardarlo como un DataFrame
* Convertir el DataFrame en un diccionario

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Crear dos DataFrame con el mapeo de codigos nace y codigos geo
mapgeo=pd.read_excel('/content/drive/MyDrive/Figaro/mapeo.xlsx','geo').set_index('cod')
mapnace=pd.read_excel('/content/drive/MyDrive/Figaro/mapeo.xlsx','nace').set_index('cod')

# Convertir los DataFrame en diccionarios
mapgeo=dict(mapgeo.to_dict()['geo'])
mapnace=dict(mapnace.to_dict()['nace'])

# Crear una tabla Figaro colapsada


* Leer la tabla figaro en formato flat.
* Crear la columna `agrega`.
* Sumar `obsValue` colapsando `agrega`
* Separar la columna `agrega` en `origen` y `destino`
* Guardar la tabla
* Crear una tabla en formato matriz
* Salvar el resultado en excel


In [ ]:
# Leer el fichero Figaro flat en un dataframe [9363300 rows × 7 columns]
ruta='/content/drive/MyDrive/Figaro/flatfile_eu-ic-io_ind-by-ind_23ed_2020.csv'
df=pd.read_csv(ruta)
df

## Crear la columna `agregar`

In [ ]:
# Crear la columna agregar
df['agrega']=df.refArea.map(mapgeo)+'_'+df.rowIi.map(mapnace)+\
  '*'+\
  df.counterpartArea.map(mapgeo)+'_'+df.colIi.map(mapnace)

In [ ]:
# Crear la columna agregar
df['agrega']=df.refArea.map(mapgeo)+'_'+df.rowIi+\
  '*'+\
  df.counterpartArea.map(mapgeo)+'_'+df.colIi

## Colapsar

Creamos una serie con los datos colapsados. El método Groupby permite crear una serie con los valores sumado por cada valor de agrega



In [ ]:
# Crear un dataframe con el campo obsValue colapsado
df=pd.DataFrame(df.groupby(['agrega'])['obsValue'].sum()).reset_index()

df

,agrega,obsValue
0,ES_A01*ES_A01,3023.615
1,ES_A01*ES_A02,9.638
2,ES_A01*ES_A03,8.399
3,ES_A01*ES_B,2.884
4,ES_A01*ES_C10T12,23736.622
...,...,...
72307,W2_OP_RES*UE_S94,0.000
72308,W2_OP_RES*UE_S95,0.000
72309,W2_OP_RES*UE_S96,0.000
72310,W2_OP_RES*UE_T,0.000


## Crear las columnas `origen` y `destino`

Los datos colapsados están en una serie cuyo indice=agrega.
A partir de esa serie creamos un DataFrame descomponiendo la agrupación nace y la agrupación georáfica

In [ ]:
# Volver a crear las columnas origen y destion a partir de la columna agrega
df['origen']=df['agrega'].str.split('*').str[0]
df['destino']=df['agrega'].str.split('*').str[1]
df

,agrega,obsValue,origen,destino
0,ES_A01*ES_A01,3023.615,ES_A01,ES_A01
1,ES_A01*ES_A02,9.638,ES_A01,ES_A02
2,ES_A01*ES_A03,8.399,ES_A01,ES_A03
3,ES_A01*ES_B,2.884,ES_A01,ES_B
4,ES_A01*ES_C10T12,23736.622,ES_A01,ES_C10T12
...,...,...,...,...
72307,W2_OP_RES*UE_S94,0.000,W2_OP_RES,UE_S94
72308,W2_OP_RES*UE_S95,0.000,W2_OP_RES,UE_S95
72309,W2_OP_RES*UE_S96,0.000,W2_OP_RES,UE_S96
72310,W2_OP_RES*UE_T,0.000,W2_OP_RES,UE_T


# Guardar los datos Figaro en un excel en formato flat y en formato matriz

El dataset está en formato largo o en formato flat, es decir, un registro para cada valor

El data set se puede salvar en excel en formato flat o en formato matriz



## Salvar la tabla en formato excel-flat

In [ ]:
df.to_excel('/content/drive/MyDrive/Figaro/agregado_flat.xlsx',index=False)

## Salvar la tabla en formato excel-matriz

Con el método `pivot`se cambia el formato del Dataframe-flat a formato DataFrame-Matriz

In [ ]:
# Pasar a formato matriz
df = df.pivot_table(index="origen", columns="destino", values="obsValue")
df

destino,ES_A01,ES_A02,ES_A03,ES_B,ES_C10T12,ES_C13T15,ES_C16,ES_C17,ES_C18,ES_C19,...,UE_P85,UE_Q86,UE_Q87_88,UE_R90T92,UE_R93,UE_S94,UE_S95,UE_S96,UE_T,UE_U
origen,,,,,,,,,,,,,,,,,,,,,
ES_A01,3023.615,9.638,8.399,2.884,23736.622,152.477,8.293,28.889,5.034,2.636,...,4.987,37.333,28.169,2.953,3.194,2.558,0.057,4.234,0.013,0.0
ES_A02,21.496,224.791,0.081,0.888,7.007,0.105,273.804,486.876,0.026,0.037,...,0.003,0.016,0.007,0.403,0.004,0.013,0.000,0.010,0.000,0.0
ES_A03,0.093,0.018,53.262,0.692,324.855,0.256,0.196,0.303,0.237,0.240,...,0.026,0.726,0.677,0.091,0.002,0.002,0.000,0.018,0.000,0.0
ES_B,3.572,0.050,2.593,10.937,18.411,0.729,0.558,22.803,0.414,394.086,...,0.155,1.512,0.315,0.047,0.073,0.045,0.004,0.185,0.000,0.0
ES_C10T12,8073.069,0.899,115.869,24.189,36350.485,293.693,25.904,170.408,46.743,5.942,...,9.515,63.419,77.075,2.716,2.911,3.676,0.208,1.980,0.001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W2_D1,5250.364,468.361,565.238,625.083,10844.767,2029.473,1299.033,1829.316,1119.048,283.869,...,360603.885,337668.346,195538.845,25652.749,22390.260,56749.528,3566.115,23188.693,29730.351,0.0
W2_D21X31,313.170,13.949,15.417,82.002,254.582,55.623,18.397,54.417,22.844,267.785,...,11140.362,24096.605,8862.180,3222.035,2438.991,4295.833,127.849,1046.338,6.927,0.0
W2_D29X39,-7092.288,4.758,-34.329,-3.969,24.361,3.519,7.032,18.399,6.396,22.610,...,129.369,-10711.293,-2733.318,-1118.505,-2158.764,-710.011,19.063,-143.183,28.110,0.0


In [ ]:
df.to_excel('/content/drive/MyDrive/Figaro/agregado_matrix.xlsx')